# Pre-process DecryptM Dataset

**Publication**: Jana Zecha et al. Decrypting drug actions and protein modifications by dose- and time-resolved proteomics.

In [1]:
import pandas as pd
import os
import toml

## Dose-Dependent Drugs

In [2]:
def search_files(directory):
    """
    Searches for .txt and .toml files in the given directory and its subdirectories.
    Args:
        directory (str): The path to the directory to search in.
    Returns:
        tuple: A tuple containing two lists:
            - txt_files (list): A list of .txt files found.
            - toml_files (list): A list of .toml files found.
    """
    txt_files = []
    toml_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                txt_files.append(file)
            elif file.endswith('.toml'):
                toml_files.append(file)

    return txt_files, toml_files


def process_experiment_column(df):
    """
    Processes the 'Experiment' column in the given DataFrame by splitting it into multiple new columns.
    Args:
        df (pd.DataFrame): The input DataFrame containing an 'Experiment' column to be processed.
    Returns:
        pd.DataFrame: The DataFrame with the 'Experiment' column split into new columns:
            - 'Type'
            - 'Cell_line'
            - 'Drug'
            - 'Time point'
            - 'Replicate'
    """
    new_columns = ["Type", "Cell_line", "Drug", "Time point", "Replicate"]
    split_columns = df['Experiment'].str.split('_', expand=True)
    num_parts = split_columns.shape[1]
    
    for col in new_columns:
        df[col] = 'NA'
    
    df[new_columns[0]] = split_columns[0]
    df[new_columns[1]] = split_columns[1]
    df[new_columns[2]] = split_columns.iloc[:, 2:num_parts-2].apply(lambda x: '_'.join(x), axis=1)
    df[new_columns[3]] = split_columns[num_parts-2]
    df[new_columns[4]] = split_columns[num_parts-1]
    
    df['Time point'] = df['Time point'].apply(lambda x: int(x[:-1]) * 60 if x.endswith('h') else int(x[:-3]) if x.endswith('min') else x)  # Convert hours to minutes and make column numeric
    
    df = df.loc[:, ~df.columns.duplicated()]
    df = df[new_columns + [col for col in df.columns if col not in new_columns]]
    
    return df


def update_dataframe_with_toml(df, toml_file, verbose = False):
    """
    Updates the column names of a DataFrame based on information from a TOML file.
    This function reads a TOML file to get dose and channel information, then renames
    the columns of the DataFrame accordingly. The columns to be renamed are expected
    to follow the pattern "TMT Channel Ratio {channel}", and they will be renamed to
    "Dose {dose}" based on the corresponding dose for each channel.
    Args:
        df (pandas.DataFrame): The DataFrame whose columns are to be renamed.
        toml_file (str): The path to the TOML file containing dose and channel information.
    Returns:
        pandas.DataFrame: The DataFrame with updated column names.
    """
    with open(toml_file, 'r') as file:
        toml_data = toml.load(file)
    doses = toml_data['TMT']['doses']
    channels = toml_data['TMT']['channels']
    dose_scale = toml_data['TMT']['dose_scale']
    dose_label = toml_data['TMT']['dose_label']
    
    if dose_scale != '1e-9' and dose_label == 'M':
        if verbose == True:
            print(f"Warning: Dose scale is {dose_scale} and dose label is {dose_label}. Multiplying doses by {float(dose_scale) / 1e-9} to convert to nM.")
        doses = [dose * float(dose_scale) / 1e-9 for dose in doses]
        
    for i, channel in enumerate(channels):
        old_column_name = f"TMT Channel Ratio {channel}"
        if old_column_name in df.columns:
            new_column_name = f"Dose {doses[i]}" if i < len(doses) else f"Dose NA"
            df.rename(columns={old_column_name: new_column_name}, inplace=True)
    return df


def filter_data(df, min_score_cutoff = 60, max_pep_cutoff = 0.05, verbose = False):
    """
    Filters the DataFrame based on the given cutoffs.
    
    Args:
        df (pd.DataFrame): The input DataFrame to be filtered.
        max_score_cutoff (float): The maximum score cutoff for filtering.
        min_pep_cutoff (float): The minimum PEP cutoff for filtering.
    
    Returns:
        pd.DataFrame: The filtered DataFrame.
    """
    rows_before = df.shape[0]

    filtered_df = df[
        (df['Max Score'] >= min_score_cutoff) & 
        (df['Min PEP'] <= max_pep_cutoff) & 
        (df['Phospho (STY)'] >= 1) & 
        (df['Phosphoproteome'] == True)
    ]

    rows_after = filtered_df.shape[0]
    if verbose == True:
        print(f"Number of rows before filtering: {rows_before}")
        print(f"Number of rows after filtering: {rows_after}")
    
    return filtered_df


def remove_columns(df):
    """
    Remove specific columns from a DataFrame.
    
    Args:
        df (pd.DataFrame): The input DataFrame from which columns will be removed.
    
    Returns:
        pd.DataFrame: The DataFrame with the specified columns removed.
    """
    columns_to_remove = [
        'Type', 'Replicate', 'N duplicates', 'Sequence', 'Length',
        'Missed cleavages', 'Proteins', 'Leading proteins', 'Protein names',
        'Phospho (STY)', 'All Phospho (STY) Probabilities', 'Max Score', 'Min PEP',
        'Intensity', 'Phosphoproteome', 'Fullproteome', 'Curve signal', 'Log EC50',
        'Curve slope', 'Curve top', 'Curve bottom', 'R2', 'Curve RMSE', 'Log EC50 error',
        'Curve slope error', 'Curve top error', 'Curve bottom error', 'EC50', 'pEC50',
        'Curve effect size', 'Regulation', 'Acetyl (K)', 'All Acetyl (K) Probabilities', 'Acetylproteome'
    ]

    df.drop(columns=columns_to_remove, inplace=True, errors='ignore')
    
    # Remove columns that start with 'Reporter intensity corrected' or 'TMT Channel Normal'
    df = df.loc[:, ~df.columns.str.startswith(('Reporter intensity corrected', 'TMT Channel Normal'))]
    
    return df


def pivot_long(df):
    """
    Converts a DataFrame from wide format to long format.
    This function takes a DataFrame with multiple columns representing different doses
    and pivots it to a long format where each row represents a single dose measurement.
    Args:
        df (pd.DataFrame): The input DataFrame in wide format.
    Returns:
        pd.DataFrame: The transformed DataFrame in long format with columns 'Dose' and 'Value'.
    """
    
    dose_columns = [col for col in df.columns if col.startswith('Dose')]
    
    df_long = pd.melt(
        df, 
        id_vars=[col for col in df.columns if col not in dose_columns], 
        value_vars=dose_columns, 
        var_name='Dose', 
        value_name='Value'
    )
    
    df_long['Dose'] = df_long['Dose'].str.replace('Dose ', '').astype(float)
    
    return df_long


def group_and_count_unique_time_points(df):
    """
    Groups the DataFrame based on the combined 'Cell_line' and 'Drug' columns and counts the unique 'Time point' values.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing 'Cell_line', 'Drug', and 'Time point' columns.
    Returns:
        pd.DataFrame: A DataFrame containing two columns:
            - 'Cell_line_Drug': The combined 'Cell_line' and 'Drug' values.
            - 'Unique Time Points': The number of unique 'Time point' values for each group.
    """
    df['CL_Drug'] = df['Cell_line'] + '_' + df['Drug']
    result_df = df.groupby('CL_Drug').agg(
        Unique_Time_Points=('Time point', 'nunique'),
        Unique_Time_Point_Values=('Time point', lambda x: list(x.unique()))
    ).reset_index()
    result_df.columns = ['CL_Drug', 'Unique Time Points', 'Unique Time Point Values']
    
    return result_df


def filter_by_unique_time_points(df, unique_timepoints_df):
    """
    Filters the original DataFrame to keep only the rows that match the groups with more than one unique time point.
    
    Args:
        df (pd.DataFrame): The original DataFrame.
        unique_timepoints_df (pd.DataFrame): The DataFrame containing the groups and their unique time points.
    
    Returns:
        pd.DataFrame: The filtered DataFrame.
    """
    filtered_groups = unique_timepoints_df[unique_timepoints_df['Unique Time Points'] > 1]
    filtered_df = df.merge(filtered_groups[['CL_Drug']], on='CL_Drug', how='inner')
    
    return filtered_df

In [3]:
def main():
    """
    Main function to preprocess and combine data from text and TOML files.
    It searches for text and TOML files in the specified directory,
    processes the text files, updates them with corresponding TOML files, filters
    the data, and combines the results into a single DataFrame.
    
    Returns:
        pd.DataFrame: A combined DataFrame containing the processed data from all
        relevant text files.
    """
    verbose = False
    directory = '../data_decrypt'
    txt_files, toml_files = search_files(directory)
    combined_df = pd.DataFrame()
    
    # Remove time-dependent file
    if 'Rituximab_td.txt' in txt_files:
        txt_files.remove('Rituximab_td.txt')
        
    for txt_file in txt_files:
        txt_file_path = os.path.normpath(directory + '/' + txt_file)
        df = pd.read_csv(txt_file_path, delimiter='\t', header=0)
        df = process_experiment_column(df)

        toml_file_name = f"pipeline_{txt_file.replace('_dd.txt', '.toml')}"
        if toml_file_name in toml_files:
            toml_file_name = os.path.normpath(directory + '/' + toml_file_name)
            df = update_dataframe_with_toml(df, toml_file_name, verbose)
        else:
            print(f"toml file: {toml_file_name} does not exist!")
            
        min_score_cutoff = 60
        max_pep_cutoff = 0.05
        df = filter_data(df, min_score_cutoff, max_pep_cutoff, verbose)
        df = remove_columns(df)
        df = pivot_long(df)  # Dose column is transformed to nM
        
        # Fix 'Carfilzomib' Drug name to not be the same with 'Bortezomib'
        if txt_file == 'Carfilzomib_dd.txt':
            df['Drug'] = df['Drug'].replace('BTZ_CFZ', 'CFZ_BTZ')
            
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    combined_df_copy = combined_df.copy()
    unique_timepoints = group_and_count_unique_time_points(combined_df_copy)
    filtered_df = filter_by_unique_time_points(combined_df_copy, unique_timepoints)
    
    return combined_df, filtered_df, unique_timepoints

if __name__ == "__main__":
    combined_df, filtered_df, unique_timepoints = main()
    display(combined_df)
    #display(unique_timepoints)

,Cell_line,Drug,Time point,Modified sequence,Experiment,Gene names,Dose,Value
0,A549,PD325901,60,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,ddPTM_A549_PD325901_60min_R1,EIF3J,10000.0,NaN
1,A549,PD325901,60,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVRK,ddPTM_A549_PD325901_60min_R1,EIF3J,10000.0,1.489595
2,A549,PD325901,60,(ac)AAAADS(ph)FSGGPAGVR,ddPTM_A549_PD325901_60min_R1,RMI2,10000.0,1.307801
3,A549,MK2206,60,(ac)AAAAPDSRVS(ph)EEENLK,ddPTM_A549_MK2206_60min_R1,RRP15,10000.0,0.789177
4,A549,PD325901,60,(ac)AAAAPDSRVS(ph)EEENLK,ddPTM_A549_PD325901_60min_R1,RRP15,10000.0,1.065007
...,...,...,...,...,...,...,...,...
9998965,HeLa,SAHA,240,YYS(ph)DSDDELTVEQR,ddPTM_HeLa_SAHA_4h_R1,BOD1L1,0.0,1.000000
9998966,HeLa,SAHA,240,YYS(ph)IDDNQNK,ddPTM_HeLa_SAHA_4h_R1,NCOA7,0.0,1.000000
9998967,HeLa,SAHA,240,YYS(ph)PCEEHPAETNQNEGSESGTIR,ddPTM_HeLa_SAHA_4h_R1,ARHGEF5,0.0,1.000000
9998968,HeLa,SAHA,240,YYSDS(ph)DDELTVEQR,ddPTM_HeLa_SAHA_4h_R1,BOD1L1,0.0,1.000000


## Rituximab Time-Dependent

In [4]:
data_file_path = os.path.join("..", "data_decrypt", "rituximab_td.txt")

rituximab_td = pd.read_csv(data_file_path, sep="\t")

In [5]:
# Filter data
# Arbitrary Cutoffs
min_score_cutoff = 60  # Confidence score for peptide identification
max_pep_cutoff = 0.05  # Minimum posterior error probability
verbose = False
rows_before = rituximab_td.shape[0]

rituximab_td = rituximab_td[
    (rituximab_td['Max Score'] >= min_score_cutoff) & 
    (rituximab_td['Min PEP'] <= max_pep_cutoff) &
    (rituximab_td['Phospho (STY)'] >= 1) &
    (rituximab_td['Phosphoproteome'] == True)
]

rows_after = rituximab_td.shape[0]

if verbose == True:
    print(f"Number of rows before filtering: {rows_before}")
    print(f"Number of rows before filtering: {rows_after}")

# Break down experiment column
split_experiment_col = rituximab_td['Experiment'].str.split('_', expand=True)
rituximab_td['Cell_line'] = split_experiment_col[1]
rituximab_td['Drug'] = split_experiment_col[2]
rituximab_td['Dose'] = split_experiment_col[3].str.replace('ng', '')

In [6]:
rituximab_td = rituximab_td[[
        'Experiment',
        'Cell_line',
        'Drug',
        'Dose',
        'Gene names',
        'Modified sequence', 
        'TMT Channel Ratio 1',
        'TMT Channel Ratio 2', 
        'TMT Channel Ratio 3', 
        'TMT Channel Ratio 4',
        'TMT Channel Ratio 5', 
        'TMT Channel Ratio 6', 
        'TMT Channel Ratio 7',
        'TMT Channel Ratio 8'
]]

rituximab_td = rituximab_td.rename(columns={
        'TMT Channel Ratio 1': '1',
        'TMT Channel Ratio 2': '2',
        'TMT Channel Ratio 3': '5',
        'TMT Channel Ratio 4': '10',
        'TMT Channel Ratio 5': '60',
        'TMT Channel Ratio 6': '120',
        'TMT Channel Ratio 7': '360',
        'TMT Channel Ratio 8': '1440'})

rituximab_td.head(10)

,Experiment,Cell_line,Drug,Dose,Gene names,Modified sequence,1,2,5,10,60,120,360,1440
0,tdPTM_Ramos_Rituximab_0ng,Ramos,Rituximab,0,PABPN1,(ac)AAAAAAAAAAGAAGGRGS(ph)GPGR,1.0,1.482082,1.304435,1.200087,1.711429,1.036134,0.939321,2.293267
1,tdPTM_SU-DHL-4_Rituximab_30000ng,SU-DHL-4,Rituximab,30000,PABPN1,(ac)AAAAAAAAAAGAAGGRGS(ph)GPGR,1.0,0.936966,0.895313,0.806706,1.030389,1.129011,0.949684,1.032427
2,tdPTM_ARH-77_Rituximab_0ng,ARH-77,Rituximab,0,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.894783,0.882104,0.860725,0.996797,0.947909,0.774303,1.274237
3,tdPTM_ARH-77_Rituximab_1000ng,ARH-77,Rituximab,1000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.925674,0.996374,1.035341,0.854041,0.649231,0.801136,1.313595
4,tdPTM_ARH-77_Rituximab_30000ng,ARH-77,Rituximab,30000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,tdPTM_Ramos_Rituximab_1000ng,Ramos,Rituximab,1000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.916323,1.344232,1.124650,1.409876,1.464369,1.204183,0.909416
6,tdPTM_Ramos_Rituximab_30000ng,Ramos,Rituximab,30000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,1.126545,0.926199,0.988858,1.031604,0.717043,0.922084,1.318389
7,tdPTM_SU-DHL-4_Rituximab_0ng,SU-DHL-4,Rituximab,0,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.840954,0.925568,1.176106,0.850982,0.765193,0.984658,0.934958
8,tdPTM_SU-DHL-4_Rituximab_1000ng,SU-DHL-4,Rituximab,1000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.651056,1.080928,0.761110,0.580987,0.492847,0.519884,0.780907
9,tdPTM_SU-DHL-4_Rituximab_30000ng,SU-DHL-4,Rituximab,30000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVRK,1.0,1.003644,1.032999,1.202614,0.978781,1.083113,0.823669,0.939890


## Combine Datasets

In [11]:
# Remove time point information from experiment column
filtered_df['Experiment'] = filtered_df['Experiment'].str.rsplit('_', n=2).str[0]
filtered_df['Time point'] = filtered_df['Time point'].astype(str)

dd_df = filtered_df.pivot(
    index=['Experiment', 'Cell_line', 'Drug', 'Dose', 'Gene names', 'Modified sequence'], 
    columns='Time point', 
    values='Value'
).reset_index()

display(dd_df)

Time point,Experiment,Cell_line,Drug,Dose,Gene names,Modified sequence,120,240,480,60,960
0,ddPTM_RPMI8226,RPMI8226,BTZ_CFZ,0.0,NaN,(ac)AEAEEDCHS(ph)DTVR,NaN,NaN,NaN,1.000000,NaN
1,ddPTM_RPMI8226,RPMI8226,BTZ_CFZ,0.0,NaN,AAS(ph)GGSGSENVGPPAAAVPGPLSR,1.000000,NaN,1.000000,1.000000,NaN
2,ddPTM_RPMI8226,RPMI8226,BTZ_CFZ,0.0,NaN,ADS(ph)SPVKAGVETTTPSK,NaN,NaN,NaN,NaN,1.000000
3,ddPTM_RPMI8226,RPMI8226,BTZ_CFZ,0.0,NaN,ADSS(ph)PVKAGVETTT(ph)PSK,NaN,NaN,NaN,NaN,1.000000
4,ddPTM_RPMI8226,RPMI8226,BTZ_CFZ,0.0,NaN,ADSS(ph)PVKAGVETTTPSK,NaN,1.000000,1.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
317899,ddPTM_RPMI8226,RPMI8226,CFZ_BTZ,1000.0,ZZZ3,VGLPARPKS(ph)PLDPK,NaN,NaN,NaN,NaN,1.106431
317900,ddPTM_RPMI8226,RPMI8226,CFZ_BTZ,1000.0,mkl1;MKL1,AGSCCLS(ph)PGGR,NaN,1.671148,1.545883,1.132695,1.306718
317901,ddPTM_RPMI8226,RPMI8226,CFZ_BTZ,1000.0,mkl1;MKL1,AYQDQIS(ph)PVPGAPK,1.621810,1.622545,NaN,NaN,NaN
317902,ddPTM_RPMI8226,RPMI8226,CFZ_BTZ,1000.0,mkl1;MKL1,FGSTGS(ph)TPPVS(ph)PTPSER,0.989267,NaN,NaN,1.348927,NaN


In [10]:
combined_df = pd.concat([rituximab_td, dd_df], axis=0, ignore_index=True, sort=False)
display(combined_df)

,Experiment,Cell_line,Drug,Dose,Gene names,Modified sequence,1,2,5,10,60,120,360,1440,240,480,960
0,tdPTM_Ramos_Rituximab_0ng,Ramos,Rituximab,0,PABPN1,(ac)AAAAAAAAAAGAAGGRGS(ph)GPGR,1.0,1.482082,1.304435,1.200087,1.711429,1.036134,0.939321,2.293267,NaN,NaN,NaN
1,tdPTM_SU-DHL-4_Rituximab_30000ng,SU-DHL-4,Rituximab,30000,PABPN1,(ac)AAAAAAAAAAGAAGGRGS(ph)GPGR,1.0,0.936966,0.895313,0.806706,1.030389,1.129011,0.949684,1.032427,NaN,NaN,NaN
2,tdPTM_ARH-77_Rituximab_0ng,ARH-77,Rituximab,0,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.894783,0.882104,0.860725,0.996797,0.947909,0.774303,1.274237,NaN,NaN,NaN
3,tdPTM_ARH-77_Rituximab_1000ng,ARH-77,Rituximab,1000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.925674,0.996374,1.035341,0.854041,0.649231,0.801136,1.313595,NaN,NaN,NaN
4,tdPTM_ARH-77_Rituximab_30000ng,ARH-77,Rituximab,30000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448808,ddPTM_RPMI8226_BTZ_CFZ,RPMI8226,CFZ_BTZ,1000.0,ZZZ3,VGLPARPKS(ph)PLDPK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.106431
448809,ddPTM_RPMI8226_BTZ_CFZ,RPMI8226,CFZ_BTZ,1000.0,mkl1;MKL1,AGSCCLS(ph)PGGR,NaN,NaN,NaN,NaN,1.132695,NaN,NaN,NaN,1.671148,1.545883,1.306718
448810,ddPTM_RPMI8226_BTZ_CFZ,RPMI8226,CFZ_BTZ,1000.0,mkl1;MKL1,AYQDQIS(ph)PVPGAPK,NaN,NaN,NaN,NaN,NaN,1.621810,NaN,NaN,1.622545,NaN,NaN
448811,ddPTM_RPMI8226_BTZ_CFZ,RPMI8226,CFZ_BTZ,1000.0,mkl1;MKL1,FGSTGS(ph)TPPVS(ph)PTPSER,NaN,NaN,NaN,NaN,1.348927,0.989267,NaN,NaN,NaN,NaN,NaN


## Exploratory Data Analysis